In [1]:
import pandas as pd
import sql_functions as sql_int
import Levenshtein

In [2]:
schema = 'tft_table'
table = 'alternative_name'
player_import = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/name_list_prismatic_legends_winter2022.csv',
                         dtype={'liquipedia_name' : str,'country_flag' : str} )
df_player_import = player_import.copy()

sql_query = f'SELECT player_id,player_name FROM {schema}.{table};'
alternative_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['player_id','player_name'])
df_alternative_name = alternative_name.copy()

## we don't take the player_identitys now to reduce the sql_querys

### Testing for duplikates in the imported Names

In [3]:
df_player_import = df_player_import.rename(columns={'liquipedia_name' : 'import_name'})
df_player_import['import_name'] = df_player_import['import_name'].str.strip()
df_player_import = df_player_import.drop_duplicates()

df_player_import[df_player_import.duplicated(keep = False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name,country_flag


In [4]:
df_player_import

,import_name,country_flag
0,Loescher,NaN
1,Bufferingo,NaN
2,GrexGoesFlex,NaN
3,Serphax,NaN
4,Vahrulv,NaN
...,...,...
188,Dunyele,NaN
189,raffipon,NaN
190,Mozart Panda,NaN
191,N8schweif,NaN


#### delete double entrys

In [5]:
df_player_import.drop([148],inplace=True)

### Testing which name is new


In [5]:
# Return only new entrys in a dataframe
def check_new(df_new,df_exist,columns):
    df_new = df_new.copy()
    df_exist = df_exist.copy()
    df_exist['already_exist'] = 'Yes'
    drop_columns = df_exist.columns.difference(df_new.columns)
    df_concat = pd.concat([df_exist,df_new],ignore_index=True)
    df_concat = (df_concat
                 .drop_duplicates(subset=[columns],keep=False)
                 .query('already_exist != "Yes"')
                 .drop(columns=drop_columns)
    )
    return df_concat


In [6]:
df_player_import = check_new(df_player_import,
                             df_alternative_name.rename(columns={'player_name' :'import_name' }),
                             'import_name')
df_player_import

,import_name,country_flag
1380,Bufferingo,NaN
1381,GrexGoesFlex,NaN
1382,Serphax,NaN
1383,Vahrulv,NaN
1384,NekoGirlLaura,NaN
...,...,...
1567,Dunyele,NaN
1568,raffipon,NaN
1569,Mozart Panda,NaN
1570,N8schweif,NaN


In [7]:
list_new_player = df_player_import['import_name'].to_list()
list_all_name = df_alternative_name['player_name'].to_list()

similar_name = []
for i in list_new_player :
    for j in list_all_name:
        if i == j:
            pass
        elif  Levenshtein.jaro(i,j) > 0.75:
            similar_name.append([j,i])
            
df_similar_name = (pd.DataFrame(similar_name,columns=['player_name','import_name']))
df_similar_name.head(50)

,player_name,import_name
0,Strouvilhou,Strohli
1,silasdvTFT,EliaTFT
2,En1gmaTFT,EliaTFT
3,HaiEloTFT,EliaTFT
4,Sameli TFT,EliaTFT
5,Leinad TFT,Lind
6,Marvelo,Mavee
7,At The Gates,AtTheGates
8,Markus000,Glurak2000
9,Tact TFT,HabichtTFT


Do some magic here

In [8]:
df_similar_name = df_similar_name.iloc[[11,14,18,26,37]]
df_similar_name

,player_name,import_name
11,CalamityHS,Calamity
14,gluteus,gluteus maximus
18,h4drauf,h4udrauf
26,TFTobby,TFTobyy
37,silasdvTFT,silasdvTFT#EUW


### Alternative_name table

In [9]:
df_similar_name = (pd.merge(df_similar_name,df_alternative_name,how='left',left_on='player_name',right_on='player_name')
                   .drop(columns=['player_name'])
                   .rename(columns={'import_name': 'player_name'}))
sql_int.push_to_database(df_similar_name,table,sql_int.get_engine_alchemy(),schema)

The alternative_name table was imported successfully.


### Player table

In [10]:
df_new_liquidpedia = (check_new(df_player_import,df_similar_name.rename(columns={'player_name':'import_name'}),'import_name')
                      .rename(columns={'import_name':'liquipedia_name','country':'country_id','country_flag':'country_id'})
                      )
table = 'player'

sql_int.push_to_database(df_new_liquidpedia,table,sql_int.get_engine_alchemy(),schema)

The player table was imported successfully.


In [11]:
sql_query = f'''
            select player_id,liquipedia_name from {schema}.{table};
            '''

df_liquipedia_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                  columns=['player_id','player_name'])

df_new_liquidpedia = (df_new_liquidpedia.rename(columns={'liquipedia_name': 'player_name'})
                      .drop(columns=['country_id'], errors='ignore'))
df_new_liquidpedia = pd.merge(df_new_liquidpedia,df_liquipedia_name,how='left')
df_new_liquidpedia.head(5)

,player_name,player_id
0,Bufferingo,1245
1,GrexGoesFlex,1246
2,Serphax,1247
3,Vahrulv,1248
4,NekoGirlLaura,1249


In [12]:

table = 'alternative_name'
sql_int.push_to_database(df_new_liquidpedia,table,sql_int.get_engine_alchemy(),schema)

The alternative_name table was imported successfully.
